In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from actor_net import ActorNet
import numpy as np
from gameworlds.hex_world import HexState

In [ ]:
with open('data/2022-03-26T22-28-10_3x3.npy', 'rb') as f:
    states = np.load(f)
    y_train = np.load(f)
    y_train_value = np.load(f)

In [ ]:
states.shape

In [ ]:

x_train = np.array(list(map(HexState.from_array_to_vector, states)))

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
anet = ActorNet()

In [ ]:
anet.model.summary()

In [ ]:
scaled = y_train ** 5

In [ ]:
scaled = scaled / np.sum(scaled, axis=1).reshape(len(scaled), 1)

In [ ]:
anet.train(x_train=x_train,y_train=scaled, y_train_value=y_train_value, epochs=100)

In [ ]:
anet.save_params(201)